In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [3]:
train_data.shape

(404, 13)

In [4]:
test_data.shape

(102, 13)

In [5]:
train_targets

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

NORMALIZING THE DATA

In [6]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

In [7]:
test_data -= mean
test_data /= std

Model definition

In [8]:
from keras import models
from keras import layers
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
    input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

K-fold validation

In [9]:
import numpy as np
k=4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

In [10]:
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate([train_data[:i * num_val_samples],train_data[(i + 1) * num_val_samples:]],axis=0)
    
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],train_targets[(i + 1) * num_val_samples:]],axis=0)
    
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)
    all_scores.append(val_mae)

processing fold # 0
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
101/101 [==============================] - 0s 1ms/step
processing fold # 1
101/101 [==============================] - 0s 1ms/step
processing fold # 2
101/101 [==============================] - 0s 2ms/step
processing fold # 3
101/101 [==============================] - 0s 2ms/step


In [11]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate([train_data[:i * num_val_samples],train_data[(i + 1) * num_val_samples:]],axis=0)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [13]:
num_epochs = 100
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
    [train_data[:i * num_val_samples],
    train_data[(i + 1) * num_val_samples:]],
    axis=0)
    partial_train_targets = np.concatenate(
    [train_targets[:i * num_val_samples],
    train_targets[(i + 1) * num_val_samples:]],
    axis=0)
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
    validation_data=(val_data, val_targets),
    epochs=num_epochs, batch_size=1, verbose=1)
    mae_history = history.history['val_mean_absolute_error']
    all_mae_histories.append(mae_history)


processing fold # 0
Train on 303 samples, validate on 101 samples
Epoch 1/100
303/303 [==============================] - 2s 8ms/step - loss: 193.7467 - mean_absolute_error: 10.1999 - val_loss: 33.2938 - val_mean_absolute_error: 3.5663
Epoch 2/100
303/303 [==============================] - 1s 3ms/step - loss: 28.2867 - mean_absolute_error: 3.6574 - val_loss: 22.3293 - val_mean_absolute_error: 3.1194
Epoch 3/100
303/303 [==============================] - 1s 3ms/step - loss: 20.8234 - mean_absolute_error: 3.0875 - val_loss: 17.3727 - val_mean_absolute_error: 2.8935
Epoch 4/100
303/303 [==============================] - 1s 3ms/step - loss: 17.9406 - mean_absolute_error: 2.8270 - val_loss: 14.7059 - val_mean_absolute_error: 2.4409
Epoch 5/100
303/303 [==============================] - 1s 3ms/step - loss: 16.4552 - mean_absolute_error: 2.6565 - val_loss: 14.1108 - val_mean_absolute_error: 2.5073
Epoch 6/100
303/303 [==============================] - 1s 3ms/step - loss: 14.1226 - mean_absolut

Epoch 50/100
303/303 [==============================] - 1s 3ms/step - loss: 6.1884 - mean_absolute_error: 1.6285 - val_loss: 9.3284 - val_mean_absolute_error: 2.0146
Epoch 51/100
303/303 [==============================] - 1s 3ms/step - loss: 6.2221 - mean_absolute_error: 1.5792 - val_loss: 8.8736 - val_mean_absolute_error: 2.1113
Epoch 52/100
303/303 [==============================] - 1s 3ms/step - loss: 6.5784 - mean_absolute_error: 1.7460 - val_loss: 8.4991 - val_mean_absolute_error: 2.1448
Epoch 53/100
303/303 [==============================] - 1s 3ms/step - loss: 6.0855 - mean_absolute_error: 1.6610 - val_loss: 8.7503 - val_mean_absolute_error: 2.0135
Epoch 54/100
303/303 [==============================] - 1s 3ms/step - loss: 6.0084 - mean_absolute_error: 1.6518 - val_loss: 10.2251 - val_mean_absolute_error: 2.2416
Epoch 55/100
303/303 [==============================] - 1s 3ms/step - loss: 6.0327 - mean_absolute_error: 1.7031 - val_loss: 9.2779 - val_mean_absolute_error: 2.1187
Epo

303/303 [==============================] - 1s 3ms/step - loss: 4.0353 - mean_absolute_error: 1.3736 - val_loss: 8.4250 - val_mean_absolute_error: 1.9715
Epoch 100/100
303/303 [==============================] - 1s 3ms/step - loss: 4.1019 - mean_absolute_error: 1.4286 - val_loss: 8.7909 - val_mean_absolute_error: 2.3181
processing fold # 1
Train on 303 samples, validate on 101 samples
Epoch 1/100
303/303 [==============================] - 2s 8ms/step - loss: 217.6070 - mean_absolute_error: 10.9763 - val_loss: 30.4277 - val_mean_absolute_error: 4.0478
Epoch 2/100
303/303 [==============================] - 1s 3ms/step - loss: 28.3122 - mean_absolute_error: 3.4499 - val_loss: 16.5687 - val_mean_absolute_error: 2.9884
Epoch 3/100
303/303 [==============================] - 1s 3ms/step - loss: 20.1126 - mean_absolute_error: 2.8020 - val_loss: 15.7574 - val_mean_absolute_error: 2.9891
Epoch 4/100
303/303 [==============================] - 1s 3ms/step - loss: 16.4821 - mean_absolute_error: 2.594

Epoch 48/100
303/303 [==============================] - 1s 3ms/step - loss: 6.6125 - mean_absolute_error: 1.6629 - val_loss: 13.0582 - val_mean_absolute_error: 2.6103
Epoch 49/100
303/303 [==============================] - 1s 3ms/step - loss: 6.7416 - mean_absolute_error: 1.6982 - val_loss: 12.8294 - val_mean_absolute_error: 2.6101
Epoch 50/100
303/303 [==============================] - 1s 3ms/step - loss: 6.2478 - mean_absolute_error: 1.6493 - val_loss: 13.7068 - val_mean_absolute_error: 2.6810
Epoch 51/100
303/303 [==============================] - 1s 3ms/step - loss: 5.9251 - mean_absolute_error: 1.6116 - val_loss: 10.4868 - val_mean_absolute_error: 2.3295
Epoch 52/100
303/303 [==============================] - 1s 3ms/step - loss: 6.2739 - mean_absolute_error: 1.6177 - val_loss: 11.6104 - val_mean_absolute_error: 2.4891
Epoch 53/100
303/303 [==============================] - 1s 3ms/step - loss: 6.3131 - mean_absolute_error: 1.6172 - val_loss: 11.8298 - val_mean_absolute_error: 2.438

303/303 [==============================] - 1s 3ms/step - loss: 3.1156 - mean_absolute_error: 1.2059 - val_loss: 13.5078 - val_mean_absolute_error: 2.6137
Epoch 98/100
303/303 [==============================] - 1s 3ms/step - loss: 3.0932 - mean_absolute_error: 1.1850 - val_loss: 15.4262 - val_mean_absolute_error: 2.6570
Epoch 99/100
303/303 [==============================] - 1s 3ms/step - loss: 3.1552 - mean_absolute_error: 1.2214 - val_loss: 14.7191 - val_mean_absolute_error: 2.6795
Epoch 100/100
303/303 [==============================] - 1s 3ms/step - loss: 3.1110 - mean_absolute_error: 1.2204 - val_loss: 13.2647 - val_mean_absolute_error: 2.6180
processing fold # 2
Train on 303 samples, validate on 101 samples
Epoch 1/100
303/303 [==============================] - 2s 7ms/step - loss: 249.2943 - mean_absolute_error: 11.8261 - val_loss: 42.3581 - val_mean_absolute_error: 4.6910
Epoch 2/100
303/303 [==============================] - 1s 3ms/step - loss: 36.4128 - mean_absolute_error: 4.0

Epoch 46/100
303/303 [==============================] - 1s 3ms/step - loss: 5.0700 - mean_absolute_error: 1.5559 - val_loss: 13.4866 - val_mean_absolute_error: 2.4853
Epoch 47/100
303/303 [==============================] - 1s 3ms/step - loss: 5.3902 - mean_absolute_error: 1.6265 - val_loss: 14.9016 - val_mean_absolute_error: 2.7325
Epoch 48/100
303/303 [==============================] - 1s 3ms/step - loss: 4.8250 - mean_absolute_error: 1.5639 - val_loss: 15.1918 - val_mean_absolute_error: 2.5505
Epoch 49/100
303/303 [==============================] - 1s 3ms/step - loss: 5.2075 - mean_absolute_error: 1.5579 - val_loss: 15.3437 - val_mean_absolute_error: 2.5906
Epoch 50/100
303/303 [==============================] - 1s 3ms/step - loss: 5.1587 - mean_absolute_error: 1.5306 - val_loss: 13.2554 - val_mean_absolute_error: 2.4783
Epoch 51/100
303/303 [==============================] - 1s 3ms/step - loss: 5.0667 - mean_absolute_error: 1.5135 - val_loss: 15.7288 - val_mean_absolute_error: 2.772

303/303 [==============================] - 1s 3ms/step - loss: 3.0834 - mean_absolute_error: 1.2369 - val_loss: 15.2870 - val_mean_absolute_error: 2.4655
Epoch 96/100
303/303 [==============================] - 1s 3ms/step - loss: 3.1847 - mean_absolute_error: 1.2570 - val_loss: 14.5203 - val_mean_absolute_error: 2.5586
Epoch 97/100
303/303 [==============================] - 1s 3ms/step - loss: 3.0416 - mean_absolute_error: 1.2198 - val_loss: 15.3708 - val_mean_absolute_error: 2.6419
Epoch 98/100
303/303 [==============================] - 1s 3ms/step - loss: 2.9594 - mean_absolute_error: 1.2623 - val_loss: 15.0801 - val_mean_absolute_error: 2.5259
Epoch 99/100
303/303 [==============================] - 1s 3ms/step - loss: 2.8474 - mean_absolute_error: 1.2299 - val_loss: 19.3351 - val_mean_absolute_error: 2.9045
Epoch 100/100
303/303 [==============================] - 1s 3ms/step - loss: 3.2837 - mean_absolute_error: 1.2702 - val_loss: 18.0927 - val_mean_absolute_error: 2.9474
processing

Epoch 44/100
303/303 [==============================] - 1s 3ms/step - loss: 6.3824 - mean_absolute_error: 1.6918 - val_loss: 13.1339 - val_mean_absolute_error: 2.4971
Epoch 45/100
303/303 [==============================] - 1s 3ms/step - loss: 6.1419 - mean_absolute_error: 1.6710 - val_loss: 13.3343 - val_mean_absolute_error: 2.5681
Epoch 46/100
303/303 [==============================] - 1s 3ms/step - loss: 6.2948 - mean_absolute_error: 1.6644 - val_loss: 12.9643 - val_mean_absolute_error: 2.4270
Epoch 47/100
303/303 [==============================] - 1s 3ms/step - loss: 6.4771 - mean_absolute_error: 1.6633 - val_loss: 14.1970 - val_mean_absolute_error: 2.7725
Epoch 48/100
303/303 [==============================] - 1s 3ms/step - loss: 6.2324 - mean_absolute_error: 1.6851 - val_loss: 13.8166 - val_mean_absolute_error: 2.4361
Epoch 49/100
303/303 [==============================] - 1s 3ms/step - loss: 6.4066 - mean_absolute_error: 1.6722 - val_loss: 14.6682 - val_mean_absolute_error: 2.605

303/303 [==============================] - 1s 3ms/step - loss: 4.1854 - mean_absolute_error: 1.3620 - val_loss: 14.2719 - val_mean_absolute_error: 2.5939
Epoch 94/100
303/303 [==============================] - 1s 3ms/step - loss: 3.9493 - mean_absolute_error: 1.3262 - val_loss: 14.6188 - val_mean_absolute_error: 2.6435
Epoch 95/100
303/303 [==============================] - 1s 3ms/step - loss: 3.9524 - mean_absolute_error: 1.2638 - val_loss: 15.0160 - val_mean_absolute_error: 2.8636
Epoch 96/100
303/303 [==============================] - 1s 3ms/step - loss: 3.8433 - mean_absolute_error: 1.3158 - val_loss: 12.7749 - val_mean_absolute_error: 2.4748
Epoch 97/100
303/303 [==============================] - 1s 3ms/step - loss: 3.8689 - mean_absolute_error: 1.3574 - val_loss: 13.1223 - val_mean_absolute_error: 2.5197
Epoch 98/100
303/303 [==============================] - 1s 3ms/step - loss: 3.8562 - mean_absolute_error: 1.2654 - val_loss: 13.1064 - val_mean_absolute_error: 2.6209
Epoch 99/10

In [ ]:
average_mae_history = [
np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]